In [ ]:
!pip install flashtext
!pip install git+https://github.com/boudinfl/pke.git
import nltk
nltk.download("punkt")
nltk.download("stopwords")
nltk.download("brown")
!pip install transformers
!pip install gensim

  Preparing metadata (setup.py) ... done
  Created wheel for flashtext: filename=flashtext-2.7-py2.py3-none-any.whl size=9296 sha256=8bb79a63c3568d0ffdc5d9ce0fc39b8761dec6c49d743b83123bfe804e4e0034
  Stored in directory: /root/.cache/pip/wheels/bc/be/39/c37ad168eb2ff644c9685f52554440372129450f0b8ed203dd
Successfully built flashtext
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-pazb4fe7
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-pazb4fe7
  Resolved https://github.com/boudinfl/pke.git to commit 69871ffdb720b83df23684fea53ec8776fd87e63
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 2.5 MB/s eta 0:00:00
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160628 sha256=5d651ea8179ece19ddd88c26b7743d975585182f1bd4192278e2334b8d68c6ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-nkum9k2h/wheels/8c/07/29/6b35bed2aa36e33d77ff3677eb71

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


In [ ]:
import string
import re
import spacy
from nltk.tokenize import sent_tokenize
from flashtext import KeywordProcessor

# Load English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load("en_core_web_sm")

def remove_subordinate_clauses(text):
    # Process the text with spaCy
    doc = nlp(text)
    # Initialize an empty list to store the main clauses
    main_clauses = []
    # Iterate through each sentence
    for sent in doc.sents:
        # Check if the sentence contains a subordinate conjunction or a dependent clause
        if any(token.dep_ == "mark" or token.dep_ == "advcl" for token in sent):
            # Remove the subordinate clause
            main_clause = ""
            for token in sent:
                if token.dep_ == "mark" or token.dep_ == "advcl":
                    break
                main_clause += token.text_with_ws
            main_clauses.append(main_clause.strip())
        else:
            main_clauses.append(sent.text)
    return main_clauses

def tokenize_sentences(text):
    """
    Tokenize the text into sentences.

    Args:
        text (str): The input text.

    Returns:
        list: List of tokenized sentences.
    """
    sentences = remove_subordinate_clauses(text)
    sentences = [sentence.strip() for sentence in sentences if len(sentence) > 20]
    return sentences

def get_named_entities(text):
    """
    Extract named entities (proper nouns, location names, country names, organization names,
    and specific common nouns) from the text using spaCy's Named Entity Recognition.

    Args:
        text (str): The input text.

    Returns:
        list: List of named entities.
    """
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    entities = set()
    for ent in doc.ents:
        if ent.label_ in ['ORG', 'LOC', 'GPE', 'PERSON']:
            entities.add(ent.text)
    return list(entities)

def get_sentences_for_keyword(keywords, sentences):
    """
    Get sentences containing the given keywords.

    Args:
        keywords (list): List of keywords.
        sentences (list): List of sentences.

    Returns:
        dict: Dictionary mapping keywords to sentences.
    """
    keyword_processor = KeywordProcessor()
    keyword_sentences = {}
    for word in keywords:
        keyword_sentences[word] = []
        keyword_processor.add_keyword(word)
    for sentence in sentences:
        keywords_found = keyword_processor.extract_keywords(sentence)
        for key in keywords_found:
            keyword_sentences[key].append(sentence)

    for key in keyword_sentences.keys():
        values = keyword_sentences[key]
        values = sorted(values, key=len, reverse=True)
        keyword_sentences[key] = values
    return keyword_sentences

def get_fill_in_the_blanks(sentence_mapping, max_questions=6):
    """
    Generate fill-in-the-blanks questions from the given sentence mapping.

    Args:
        sentence_mapping (dict): Dictionary mapping keywords to sentences.
        max_questions (int): Maximum number of questions to generate.

    Returns:
        dict: Fill-in-the-blanks questions with matching words at the top.
    """
    out = {"Q)": "Q) Fill in the blanks with matching words at the top : "}
    blank_sentences = []
    keys = []
    used_sentences = set()
    question_count = 0
    for key in sentence_mapping:
        if len(sentence_mapping[key]) > 0:
            for sent in sentence_mapping[key]:
                if sent not in used_sentences and key.capitalize() not in keys:
                    insensitive_sent = re.compile(re.escape(key), re.IGNORECASE)
                    no_of_replacements = len(re.findall(re.escape(key), sent, re.IGNORECASE))
                    line = insensitive_sent.sub(' _____________ ', sent)
                    if no_of_replacements < 2:
                        # Ensure the last symbol is a full stop
                        if line[-1] not in string.ascii_letters:
                            line = line.rstrip(string.punctuation) + "."
                        blank_sentences.append(line)
                        keys.append(key.capitalize())  # Capitalizing the options
                        used_sentences.add(sent)
                        question_count += 1
                        if question_count >= max_questions:
                            break
        if question_count >= max_questions:
            break
    # Remove spaces around each item in Options and join them with commas
    out["Options"] = keys[:10]
    # Remove the comma between questions
    out["Questions"] = blank_sentences[:10]

    return out

text = """
In the realm of Indian cinema, the 1990s marked an era of unparalleled achievements and cinematic marvels.
From the iconic films that graced the silver screen to the unforgettable performances that captivated audiences worldwide, Bollywood witnessed a golden age of creativity and innovation.
Under the visionary direction of luminaries like Yash Chopra and Subhash Ghai, Bollywood soared to new heights, producing timeless classics that continue to resonate with audiences to this day.
Superstars such as Shah Rukh Khan, Madhuri Dixit, and Aamir Khan emerged as the reigning icons of Indian cinema, enchanting viewers with their charisma and versatility.
Filming locations ranging from the picturesque valleys of Switzerland to the bustling streets of Mumbai provided the perfect backdrop for these cinematic masterpieces, transporting audiences to enchanting worlds of romance, drama, and intrigue.
The 1990s also witnessed the rise of groundbreaking films like "Dilwale Dulhania Le Jayenge" and "Hum Aapke Hain Koun..!", which redefined the concept of blockbuster cinema and set new benchmarks for success in the industry.
With its rich tapestry of stories, unforgettable music, and larger-than-life characters, the Bollywood of the 1990s remains an integral part of India's cultural heritage, continuing to inspire generations of filmmakers and audiences alike.
"""

sentences = tokenize_sentences(text)
named_entities = get_named_entities(text)
keyword_sentence_mapping = get_sentences_for_keyword(named_entities, sentences)
fill_in_the_blanks = get_fill_in_the_blanks(keyword_sentence_mapping)

# Printing the fill-in-the-blanks questions without JSON formatting
print(fill_in_the_blanks["Q)"])
print("\n")
print(fill_in_the_blanks["Options"])
print("\n")
print("\n".join([f"{i + 1}) {question}" for i, question in enumerate(fill_in_the_blanks["Questions"])]))


Q) Fill in the blanks with matching words at the top : 


['Hain koun', 'Subhash ghai', 'Switzerland', 'Aamir khan', 'India', 'Bollywood']


1) The 1990s also witnessed the rise of groundbreaking films like "Dilwale Dulhania Le Jayenge" and "Hum Aapke  _____________ .
2) Under the visionary direction of luminaries like Yash Chopra and  _____________ , Bollywood soared to new heights.
3) Filming locations ranging from the picturesque valleys of  _____________  to the bustling streets of Mumbai provided the perfect backdrop for these cinematic masterpieces.
4) Superstars such as Shah Rukh Khan, Madhuri Dixit, and  _____________  emerged as the reigning icons of Indian cinema.
5) With its rich tapestry of stories, unforgettable music, and larger-than-life characters, the Bollywood of the 1990s remains an integral part of  _____________ 's cultural heritage.
6) From the iconic films that graced the silver screen to the unforgettable performances that captivated audiences worldwide,  ______